# Interpolation in SE(3) using Lie-Group Theory Using `PyLie`


Install libraries

In [2]:
!pip -q install https://github.com/tussedrotten/pylie/archive/master.zip
!pip -q install https://github.com/tussedrotten/visgeom/archive/master.zip

     - 33.7 kB 98.2 MB/s 0:00:00
  Preparing metadata (setup.py) ... done
     - 9.5 kB ? 0:00:00
  Preparing metadata (setup.py) ... done


# Interpolation in SE(3) using Lie-Group Theory

These notes describe the use of Lie-Group Theory to perform interpolations of poses in $SE(3)$. We begin by defining the minimal element in the tangent space $\mathfrak{se}(3)$, which is called the *twist vector*. We then describe how interpolation is done using Lie-Group Theory, implemented using the `pylie` library.

The Lie-Group Theory concepts described are based on the paper by Sola et al. (2018).

---
## The twist vector

Let us start by defining the twist, which is an element of the Lie Algebra (i.e., the tangent space of the Lie Group SE(3) calculated at its identity element).

> A twist vector is the Lie-algebra representation of rigid-body motion, encoding translation and rotation together as a single 6-D vector whose exponential map produces an element of $SE(3)$.

A twist is a 6-vector:
$$
\boldsymbol{\xi}
=
\begin{bmatrix}
\boldsymbol{\rho} \\
\boldsymbol{\phi}
\end{bmatrix}
\in \mathbb{R}^6,
$$
consisting of two parts:

- $\boldsymbol{\phi} \in \mathbb{R}^3$ — **angular part** (rotation vector, axis–angle)
- $\boldsymbol{\rho} \in \mathbb{R}^3$ — **linear part** (translation coupled to rotation)

Together, $\boldsymbol{\xi}$ parameterizes an element of $\mathfrak{se}(3)$.


### The twist in matrix (hat) form

Using the hat operator $(\cdot)^\wedge$, the matrix form of the twist is given by:
$$
\boldsymbol{\xi}^\wedge
=
\begin{bmatrix}
[\boldsymbol{\phi}]_\times & \boldsymbol{\rho} \\
\mathbf{0}^\top & 0
\end{bmatrix},
$$
with $[\boldsymbol{\phi}]_\times$ the skew-symmetric matrix of $\boldsymbol{\phi}$.

### How a twist moves a pose

A finite rigid-body motion is obtained via the exponential map:
$$
T = \exp(\boldsymbol{\xi}^\wedge) \in SE(3).
$$

- **Small $\boldsymbol{\xi}$** → small motion (first-order approximation)
- **Large $\boldsymbol{\xi}$** → finite rigid transform through $\exp$

A twist represents a **screw motion**, i.e., a simultaneous rotation about an axis and translation along or around that axis (rotation + translation). Special cases include the pure translation and pure rotation:

- $\boldsymbol{\phi}=\mathbf{0}$ → **pure translation**
- $\boldsymbol{\rho}=\mathbf{0}$ → **pure rotation**

### Relation to velocity (continuous time)

If a rigid body has angular velocity $\boldsymbol{\omega}$ and linear velocity $\mathbf{v}$ then the **body twist** is:
$$
\boldsymbol{\xi} =
\begin{bmatrix}
\mathbf{v} \\
\boldsymbol{\omega}
\end{bmatrix},
\qquad
\dot{T} = T\,\boldsymbol{\xi}^\wedge.
$$


---

## Interpolation in SE(3)

Now, that we have defined the twist vector, we proceed with description of the interpolation procedure for pose frames. In this section, we will also briefly define the Lie-group operators *oplus* $\oplus$ and *ominus* $\ominus$.  

### Interpolation of poses using the `pylie` library

When using `pylie`, we can interpolate poses as if we were doing linear interpolation:

```python
 def interpolate_lie_element(alpha, X_1, X_2):
    return X_1 + alpha * (X_2 - X_1)
```

> In `pylie`, the expression $X_1 + \alpha (X_2 - X_1)$  is shorthand for  $X_1 \oplus \alpha (X_2 \ominus X_1)$, which is a **log–exp geodesic interpolation on SE(3)**.



In reality, pose elements $X_1, X_2 \in SE(3)$, live on a smooth manifold and cannot be correctly interpolated as if they were in Euclidean space.  So, the following interpolation:
$$
Y = X_1 + \alpha  (X_2 - X_1),
$$
if done directly in Euclidean space would be incorrect because $SE(3)$ is not a vector space. Instead, we should use the Lie-group operators *oplus* $\oplus$ and *ominus* $\ominus$. These operators are represented in the `pylie` library by overloading the `+` and `-` operations, respectively.  

Next, we describe each component operation of the interpolation in Equation $5$.

### The *ominus* operator ($\ominus$) : Relative pose (or residual pose) in the Lie algebra

For pose elements $X_1, X_2 \in SE(3)$, when using `pylie` , we can write their relative pose (residual pose) as we would do in Euclidean space:
$$
\xi = X_2 - X_1
$$
The library will then calculate the residual in the Lie Algebra as:
$$
\xi = X_2 \,\ominus\, X_1
\;\;\triangleq\;\;
\log\!\left( X_1^{-1} X_2 \right)
\;\in\;
\mathfrak{se}(3).
$$
This is the *logarithmic map* from the group to its tangent space.

### Scalar multiplication in the Lie algebra

Given $\xi \;\in\; \mathfrak{se}(3)$, the scalar multiplication is:
$$
\alpha \, \xi \;\in\; \mathfrak{se}(3),
$$
which scales a twist vector in the tangent space. Scaling by $\alpha$ moves the twist **linearly in the tangent space**. Because the Lie algebra $\mathfrak{se}(3)$ is a **vector space**,

- vectors add linearly
- scalars multiply linearly
- straight lines are well-defined

As a result, the path:
$$
\xi(\alpha) = \alpha \xi
$$
is a **straight line** from $0$ to $\xi$ in a Euclidean space. A straight line in tangent space becomes a **curved path** on the manifold (i.e., the geodesic). With the scaling, we have two paths depending on how we look at it: we have the **twist path**, which is linear, and the pose path, which is curved (shortest-path on the manifold).

### The *oplus* operator ($\oplus$) : Retraction back to the manifold

>  The $\oplus$ operator is a user-defined retraction that applies a tangent-space increment to a pose, typically defined as $X \oplus \xi = X \exp(\xi)$. In the context of pose trajectories, we can think of it as the operator that applies a small tangent-space increment to a pose.

In `pylie`, the `+` symbol has been overloaded, and we can write the incremental change to the pose as follows:

```python
X_1 + ξ
```

This operation corresponds to:
$$
X_1 \,\oplus\, \xi
\;\;\triangleq\;\;
X_1 \exp(\xi),
$$
which maps a tangent-space increment back onto the manifold.

### Summary

In summary, in `pylie`, given the declared objects `X_1 = SE3()` and  `X_2 = SE3()`, the following line:

```python
X_1 + alpha * (X_2 - X_1)
```

is mathematically equivalent to:
$$
X(\alpha)
=
X_1 \;\oplus\;
\alpha \big( X_2 \,\ominus\, X_1 \big)
=
X_1
\exp\!\left(
\alpha \,
\log\!\left(
X_1^{-1} X_2
\right)
\right)
$$
This approach results in a proper geodesic interpolation on SE(3) which respects rigid-body geometry. There are no Euler-angle artifacts and it always produces valid SE(3) elements. The resulting trajectory jointly interpolates rotation and translation.

## References

J. Sola, J. Deray, and D. Atchuthan, *A micro Lie theory for state estimation in robotics*, arXiv:1812.01537, 2018.



In [3]:
#@title # Pose interpolation on SE(3)
#@markdown Move slider to see interpolated elements along the path.
# ============================================================
# Plotly notebook version: pose interpolation on SE(3)
# ============================================================

import numpy as np
import plotly.graph_objects as go
from pylie import SO3, SE3

fixed_camera = dict(
    eye=dict(x=1.8, y=1.8, z=1.2),
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
)


# ------------------------------------------------------------
# Helper: linear interpolation in the Lie algebra
# (same as your original code)
# ------------------------------------------------------------
def interpolate_lie_element(alpha, X_1, X_2):
    return X_1 + alpha * (X_2 - X_1)

# ------------------------------------------------------------
# Helper: plot a coordinate frame (Plotly version of vg.plot_pose)
# ------------------------------------------------------------
def plotly_frame(data, R, t, scale=1.0, opacity=1.0):
    t = t.flatten()
    colors = ["red", "green", "blue"]
    for i in range(3):
        axis = t + scale * R[:, i]
        data.append(
            go.Scatter3d(
                x=[t[0], axis[0]],
                y=[t[1], axis[1]],
                z=[t[2], axis[2]],
                mode="lines",
                line=dict(color=colors[i], width=6),
                opacity=opacity,
                showlegend=False,
            )
        )

# ------------------------------------------------------------
# Define the two poses (same as original)
# ------------------------------------------------------------
T_1 = SE3((
    SO3.from_roll_pitch_yaw(np.pi / 4, 0, np.pi / 2),
    np.array([[1, 1, 1]]).T
))

T_2 = SE3((
    SO3.from_roll_pitch_yaw(-np.pi / 6, np.pi / 4, np.pi / 2),
    np.array([[1, 4, 2]]).T
))

# ------------------------------------------------------------
# Precompute interpolation frames (Plotly animation model)
# ------------------------------------------------------------
alphas = np.linspace(0, 1, 21)
frames = []

for a in alphas:
    data = []

    # Plot the two endpoint poses
    plotly_frame(data, *T_1.to_tuple(), scale=1.0)
    plotly_frame(data, *T_2.to_tuple(), scale=1.0)

    # Interpolated pose
    T = interpolate_lie_element(a, T_1, T_2)
    plotly_frame(data, *T.to_tuple(), scale=1.0, opacity=0.25)

    frames.append(
        go.Frame(
            data=data,
            name=f"{a:.2f}"
        )
    )

# ------------------------------------------------------------
# Base figure
# ------------------------------------------------------------
fig = go.Figure(
    data=frames[0].data,
    frames=frames
)

# ------------------------------------------------------------
# Slider (animation-based, notebook-safe)
# ------------------------------------------------------------
fig.update_layout(
    sliders=[{
        "steps": [
            {
                "args": [[f"{a:.2f}"], {"frame": {"duration": 0}, "mode": "immediate"}],
                "label": f"{a:.2f}",
                "method": "animate",
            }
            for a in alphas
        ],
        "currentvalue": {"prefix": "α = "},
        "pad": {"t": 30},
    }],
    scene=dict(
        aspectmode="data",
        xaxis=dict(range=[0, 4]),
        yaxis=dict(range=[0, 5]),
        zaxis=dict(range=[0, 4]),
        camera=fixed_camera,   # 🔒 lock camera
    ),
    title="Linear Interpolation of Poses on SE(3)",
    margin=dict(l=0, r=0, t=50, b=0),
)

fig


In [4]:
#@title # Pose interpolation on SE(3) displaying path using transparency
#@markdown Move slider to see interpolated elements along the path.
# ============================================================
# Plotly notebook version: pose interpolation on SE(3)
# ============================================================


# ============================================================
# Plotly notebook version: pose interpolation on SE(3)
# with full interpolation path
# ============================================================

import numpy as np
import plotly.graph_objects as go
from pylie import SO3, SE3

# ------------------------------------------------------------
# Linear interpolation (same as original example)
# ------------------------------------------------------------
def interpolate_lie_element(alpha, X_1, X_2):
    return X_1 + alpha * (X_2 - X_1)

# ------------------------------------------------------------
# Plotly version of vg.plot_pose
# ------------------------------------------------------------
def plotly_frame(data, R, t, scale=1.0, opacity=1.0):
    t = t.flatten()
    colors = ["red", "green", "blue"]
    for i in range(3):
        axis = t + scale * R[:, i]
        data.append(
            go.Scatter3d(
                x=[t[0], axis[0]],
                y=[t[1], axis[1]],
                z=[t[2], axis[2]],
                mode="lines",
                line=dict(color=colors[i], width=6),
                opacity=opacity,
                showlegend=False,
            )
        )

# ------------------------------------------------------------
# Define the two poses (same as original)
# ------------------------------------------------------------
T_1 = SE3((
    SO3.from_roll_pitch_yaw(np.pi / 4, 0, np.pi / 2),
    np.array([[1, 1, 1]]).T
))

T_2 = SE3((
    SO3.from_roll_pitch_yaw(-np.pi / 6, np.pi / 4, np.pi / 2),
    np.array([[1, 4, 2]]).T
))

# ------------------------------------------------------------
# Precompute the interpolation path (ALL frames)
# ------------------------------------------------------------
alphas = np.linspace(0, 1, 21)
interp_poses = [interpolate_lie_element(a, T_1, T_2) for a in alphas]

# ------------------------------------------------------------
# Build animation frames
# ------------------------------------------------------------
frames = []

for k, a in enumerate(alphas):
    data = []

    # Endpoint poses
    plotly_frame(data, *T_1.to_tuple(), scale=1.0)
    plotly_frame(data, *T_2.to_tuple(), scale=1.0)

    # Full interpolation path (faint)
    for T in interp_poses:
        plotly_frame(data, *T.to_tuple(), scale=0.8, opacity=0.3)

    # Highlight current interpolated pose
    plotly_frame(data, *interp_poses[k].to_tuple(), scale=1.0, opacity=0.9)

    frames.append(
        go.Frame(
            data=data,
            name=f"{a:.2f}"
        )
    )

# ------------------------------------------------------------
# Base figure
# ------------------------------------------------------------
fig = go.Figure(
    data=frames[0].data,
    frames=frames
)

# ------------------------------------------------------------
# Slider (animation-based, notebook-safe)
# ------------------------------------------------------------
fig.update_layout(
    sliders=[{
        "steps": [
            {
                "args": [[f"{a:.2f}"], {"frame": {"duration": 0}, "mode": "immediate"}],
                "label": f"{a:.2f}",
                "method": "animate",
            }
            for a in alphas
        ],
        "currentvalue": {"prefix": "α = "},
        "pad": {"t": 30},
    }],
    scene=dict(
        aspectmode="data",
        xaxis=dict(range=[0, 4]),
        yaxis=dict(range=[0, 5]),
        zaxis=dict(range=[0, 4]),
    ),
    title="Pose Interpolation on SE(3) with Full Path",
    margin=dict(l=0, r=0, t=50, b=0),
)

fig


# Interpolation Methods on $SE(3)$

This section contrasts two conceptually different approaches to interpolating
rigid-body poses on the Special Euclidean group $SE(3)$: *tangent-space
interpolation* and *manifold (geodesic) interpolation*. Although both
methods make use of the Lie algebra $\mathfrak{se}(3)$, only the latter produces
a geometrically and physically meaningful interpolation.

---

## Tangent-Space Interpolation (Linear in $\mathfrak{se}(3)$)

A naive approach to interpolating poses would first map each end pose
independently to the Lie algebra using the logarithmic map, interpolate
linearly in the algebra, and then map back to the group using the exponential
map.

Let $T_1, T_2 \in SE(3)$. The method is defined as:

$$
\xi_1 = \log(T_1), \qquad
\xi_2 = \log(T_2),
$$

$$
\xi(\alpha) = (1-\alpha)\,\xi_1 + \alpha\,\xi_2,
$$

$$
T_{\text{lin}}(\alpha) =
\exp\!\big(\xi(\alpha)\big), \qquad \alpha \in [0,1].
$$

This procedure interpolates along a straight line in the vector space
$\mathfrak{se}(3)$, which is the tangent space of $SE(3)$ at the identity.

There are a number of limitations in interpolating poses along a straight line
in the vector space. Although mathematically well defined, this interpolation is
generally not meaningful on $SE(3)$ due to the following reasons:

- The Lie algebra $\mathfrak{se}(3)$ is only a local linearization at the identity.
- Absolute pose logarithms $\log(T_1)$ and $\log(T_2)$ do not encode relative motion.
- The resulting path depends on the choice of reference frame.
- The interpolated trajectory is not a geodesic and has no direct physical interpretation.

As a result, tangent-space interpolation is only valid for very small motions
near the identity and should not be used for pose trajectories or visualization.

---

## Manifold (Geodesic) Interpolation on $SE(3)$

We can calculate a geometrically correct interpolation by first computing the
*relative pose* between two poses and then applying a scaled version of
this relative motion.

Using the Lie-group operators $\ominus$ and $\oplus$, the interpolation is
defined as:

$$
\delta = T_2 \ominus T_1
= \log\!\left(T_1^{-1} T_2\right)
\in \mathfrak{se}(3),
$$

$$
T(\alpha) = T_1 \oplus \alpha \delta
= T_1 \exp(\alpha \delta),
\qquad \alpha \in [0,1].
$$

In this method, the relative motion $\delta$ is expressed in the tangent space
attached to $T_1$. Scaling by $\alpha$ moves linearly in the Lie algebra, and the
exponential map retracts the scaled motion back onto the manifold.

This interpolation method produces the geodesic between $T_1$ and $T_2$ under
the canonical left-invariant metric, corresponds to motion under a constant
body-frame twist, always yields valid elements of $SE(3)$, and is invariant to
the choice of coordinate frame.

Table 1 lists the differences between the two methods.

---

### Comparison between the two approaches to interpolation in $SE(3)$

| Aspect                     | Tangent-Space          | Manifold (Geodesic)       |
|----------------------------|------------------------|---------------------------|
| Interpolation domain       | Fixed $\mathfrak{se}(3)$ | Moving tangent space      |
| Uses relative motion       | No                     | Yes                       |
| Coordinate-invariant       | No                     | Yes                       |
| Geodesic on $SE(3)$        | No                     | Yes                       |
| Physical interpretation    | None                   | Constant twist motion     |
| Suitable for trajectories  | No                     | Yes                       |

---

Tangent-space interpolation should be avoided except as a local approximation
near the identity. For all practical applications involving rigid-body motion,
such as visualization, trajectory generation, and filtering, manifold
interpolation via the log–exp construction must be used.


In [5]:
#@title # Tangent-space vs Manifold interpolation on SE(3)
#@markdown A comparison between two methods of interpolation The correct method
#@markdown is the manifold interpolation because the tangent-space interpolation
#@markdown approximates the correct interpolation near the origin.

# ============================================================
# Tangent-space vs Manifold interpolation on SE(3)
# ============================================================

import numpy as np
import plotly.graph_objects as go
from pylie import SO3, SE3

# ------------------------------------------------------------
# Helper: Plotly version of vg.plot_pose (FIXED)
# ------------------------------------------------------------
def plotly_frame(fig, R, t, scale=0.6, opacity=1.0):
    t = t.flatten()
    colors = ["red", "green", "blue"]
    for i in range(3):
        axis = t + scale * R[:, i]
        fig.add_trace(
            go.Scatter3d(
                x=[t[0], axis[0]],
                y=[t[1], axis[1]],
                z=[t[2], axis[2]],
                mode="lines",
                line=dict(color=colors[i], width=5),
                opacity=opacity,
                showlegend=False,
            )
        )

# ------------------------------------------------------------
# Define two poses
# ------------------------------------------------------------
T1 = SE3((
    SO3.from_roll_pitch_yaw(np.pi/4, 0, np.pi/2),
    np.array([[1, 1, 1]]).T
))

T2 = SE3((
    SO3.from_roll_pitch_yaw(-np.pi/6, np.pi/4, np.pi/2),
    np.array([[4, 3, 2]]).T
))

# ------------------------------------------------------------
# Interpolation parameters
# ------------------------------------------------------------
alphas = np.linspace(0, 1, 15)

# ------------------------------------------------------------
# Build figure
# ------------------------------------------------------------
fig = go.Figure()

# Start and end poses
plotly_frame(fig, *T1.to_tuple(), scale=1.0, opacity=1.0)
plotly_frame(fig, *T2.to_tuple(), scale=1.0, opacity=1.0)

# ------------------------------------------------------------
# 1️⃣ Tangent-space interpolation (WRONG path)
# ------------------------------------------------------------
xi1 = T1.Log()
xi2 = T2.Log()

# Path on the tangent space (linear)
for a in alphas:
    xi_lin = (1 - a) * xi1 + a * xi2
    T_lin = SE3.Exp(xi_lin)
    plotly_frame(fig, *T_lin.to_tuple(), scale=0.5, opacity=0.15)

# ------------------------------------------------------------
# 2️⃣ Manifold interpolation (CORRECT path) (geodesic)
# ------------------------------------------------------------
delta = T2 - T1  # Log(T1^{-1} T2)

for a in alphas:
    T_geo = T1 + a * delta
    plotly_frame(fig, *T_geo.to_tuple(), scale=0.5, opacity=0.6)

# ------------------------------------------------------------
# Layout
# ------------------------------------------------------------
fig.update_layout(
    scene=dict(
        aspectmode="data",
        xaxis=dict(range=[0, 5]),
        yaxis=dict(range=[0, 5]),
        zaxis=dict(range=[0, 4]),
    ),
    title=(
        "Tangent-space (faint) vs Manifold (bold) interpolation on SE(3)<br>"
        "<span style='font-size:14px'>"
        "Faint = linear in se(3), Bold = geodesic on SE(3)</span>"
    ),
    margin=dict(l=0, r=0, t=70, b=0),
)

fig
